<a href="https://colab.research.google.com/github/jenferon/Machine_Learning_Practise/blob/main/initial_notebooks_2_practise/generate_21cm_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Diffusion model testing


In [ ]:
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import struct
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim import Adam
import functools

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, base, data_name, transform=None, target_transform=None):
       self.transform = transform
       self.target_transform = target_transform
       self.image_loc = os.path.join(base, data_name)
       # Open the file in binary read mode ('rb')
       self.fid = open(self.image_loc, 'rb')
       self.dtype_size = 4  # Assuming float32 data type

    def __len__(self):
       # Get file size and calculate number of images
       file_size = os.path.getsize(self.image_loc)
       num_images = file_size // (600 * 600 * self.dtype_size)
       return num_images # Corrected length calculation

    def __getitem__(self, idx):
        # Calculate the starting position for the slice
        offset = idx * 600 * 600 * self.dtype_size

        # Seek to the correct position in the file
        self.fid.seek(offset)

        # Read only the required slice
        data = self.fid.read(600 * 600 * self.dtype_size)

        # Unpack the data for the slice into a NumPy array
        image = struct.unpack('f' * (600 * 600), data)
        image = np.array(image).reshape(600, 600)
        image = image[:256,:256]
        # Convert to PyTorch tensor and add channel dimension
        image = torch.from_numpy(image).float().unsqueeze(0)

        label = 'z=12.0'
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

    def __del__(self):
        # Close the file when the dataset object is deleted
        self.fid.close()

In [ ]:
def marginal_prob_std(t, sigma):
  """Compute the mean and standard deviation of $p_{0t}(x(t) | x(0))$.

  Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

  Returns:
    The standard deviation.
  """
  t = torch.tensor(t, device=device)
  return torch.sqrt((sigma**(2 * t) - 1.) / 2. / np.log(sigma))

def loss_fn(model, x, marginal_prob_std, eps=1e-5):
  """The loss function for training score-based generative models.

  Args:
    model: A PyTorch model instance that represents a
      time-dependent score-based model.
    x: A mini-batch of training data.
    marginal_prob_std: A function that gives the standard deviation of
      the perturbation kernel.
    eps: A tolerance value for numerical stability.
  """
  random_t = torch.rand(x.shape[0], device=x.device) * (1. - eps) + eps
  z = torch.randn_like(x)
  std = marginal_prob_std(random_t)
  perturbed_x = x + z * std[:, None, None, None]
  score = model(perturbed_x, random_t)
  loss = torch.mean(torch.sum((score * std[:, None, None, None] + z)**2, dim=(1,2,3)))
  return loss

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/ML_Prac/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-6954684fc411>", line 1, in <cell line: 0>
    get_ipython().run_line_magic('cd', '/content/gdrive/MyDrive/ML_Prac/')
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurre

In [ ]:
ls

deltaTb_z12.000_N600_L200.0.dat  Untitled0.ipynb  Untitled1.ipynb


In [ ]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer

ModuleNotFoundError: No module named 'denoising_diffusion_pytorch'

In [ ]:
sigma =  25.0
marginal_prob_std_fn = functools.partial(marginal_prob_std, sigma=sigma)

batch_size = 5

dataset = CustomImageDataset('/content/gdrive/MyDrive/ML_Prac/','deltaTb_z12.000_N600_L200.0.dat')
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
score_model = torch.nn.DataParallel(ScoreNet(marginal_prob_std=marginal_prob_std_fn))
score_model = score_model.to(device)

optimizer = Adam(score_model.parameters(), lr=1e-4)

tqdm_epoch = tqdm.trange(10)
for epoch in tqdm_epoch:
  avg_loss = 0.
  num_items = 0
  for x, y in data_loader:
    x = x.to(device)
    loss = loss_fn(score_model, x, marginal_prob_std_fn)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    avg_loss += loss.item() * x.shape[0]
    num_items += x.shape[0]
  # Print the averaged training loss so far.
  tqdm_epoch.set_description('Average Loss: {:5f}'.format(avg_loss / num_items))

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-16-8a4856826ccb>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t, device=device)
  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 60 but got size 62 for tensor number 1 in the list.